In [1]:
### imports
import json
from copy import deepcopy

from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))

In [3]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
print(len(medDocs))
medDocs[1]

59


{'articleId': 'PMC7282151',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282151/',
 'articleTitle': 'SPONTANEOUS CEREBROSPINAL FLUID LEAK AFTER INITIATION OF DOPAMINE AGONIST THERAPY IN MACROPROLACTINOMAS: TWO CASE REPORTS AND A LITERATURE REVIEW',
 'image_path': ['PMC7282151-1_i2376-0605-6-2-e90-f01.jpg',
  'PMC7282151-2_i2376-0605-6-2-e90-f02.jpg',
  'PMC7282151-3_i2376-0605-6-2-e90-f03.jpg'],
 'image_url': ['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282151/bin/i2376-0605-6-2-e90-f01.jpg',
  'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282151/bin/i2376-0605-6-2-e90-f02.jpg',
  'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282151/bin/i2376-0605-6-2-e90-f03.jpg'],
 'caption': ['Magnetic resonance images of case 1 at admission showing macroprolactinoma occupying the sella in sagittal (left) and coronal (right) views.',
  'Coronal post-contrast images of case 2 showing T1 (left) and T2 (right) views of the mass extending to the optic chiasm with bilateral cavernou

In [14]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
#         cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        tokens += sentTokens2
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = deepcopy(medDocs)
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        for j, cap in enumerate(doc['caption']):
            doc['caption'][j] = preprocessContent(cap)
    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

___________________________________________________
Axial contrast-enhanced computed tomography scans prelenvatinib therapy (top left), 4 months postlenvatinib therapy (top middle), and 5 months post-surgery (top right). Note the 270-degree encirclement of the trachea by the left thyroid mass with potential membranous trachea and esophageal invasion prior to lenvatinib therapy (top left) timeline (middle) calcitonin and CEA concentrations versus time (bottom). CEA = carcinoembryonic antigen US = ultrasound.
Axial contrast-enhanced computed tomography scans prelenvatinib therapy (top left), 4 months postlenvatinib therapy (top middle), and 5 months post-surgery (top right). Note the 270-degree encirclement of the trachea by the left thyroid mass with potential membranous trachea and esophageal invasion prior to lenvatinib therapy (top left) timeline (middle) calcitonin and CEA concentrations versus time (bottom). CEA = carcinoembryonic antigen US = ultrasound.
__________________________

___________________________________________________
Prediction-corrected visual predictive check (upper panels), and representative individual time course of platelet counts and the corresponding model predictions for non-splenectomized (middle panels) and splenectomized patients (lower panels) stratified by cohort
Prediction-corrected visual predictive check (upper panels), and representative individual time course of platelet counts and the corresponding model predictions for non-splenectomized (middle panels) and splenectomized patients (lower panels) stratified by cohort
___________________________________________________
Influence of surgery-specific parameters SPmax and k p on platelet profiles
Influence of surgery-specific parameters SPmax and k p on platelet profiles
___________________________________________________
Effect of initial oxaliplatin concentrations on the peritoneum, treatment duration, and carrier solution on platelet dynamics in non-splenectomized and splenectom

The final population pharmacokinetic model normalized prediction distribution error (NPDE) results for morphine (upper row), M3G (middle row), and M6G (lower row). The histograms show the NPDE distribution, and the solid lines represent the normal distribution. The mean and variance of the NPDE are shown below each histogram. Asterisk indicates significant difference of the mean from 0 and the variance from 1 at the pвЂ‰<вЂ‰0.05 level, using StudentвЂ™s t test for mean and Fisher test for variance. The scatter plots show the distribution of NPDE versus time since last dose and population predicted concentrations (on logarithmic scale). The dotted lines represent the NPDE 5th and 95th percentiles. Zero line (dashed) and LOWESS smoother (solid) are shown
___________________________________________________
Box plot of the simulation results following the administration of 0.1В mg/kg morphine every 3В h showing morphine AUC0вЂ“24В h in 1-month-, 6-month-, and 2-year-old children, M3G, and 

Metabolic profiles of FDU-PB-22 (a) and FUB-PB-22 (b) after 3-h incubation in human hepatocytes. The inserts are the expanded chromatograms of early eluted minor metabolites
___________________________________________________
aвЂ“h Product ion spectra, proposed structures, and fragmentation patterns of FDU-PB-22, FUB-PB-22, and their metabolites
aвЂ“h Product ion spectra, proposed structures, and fragmentation patterns of FDU-PB-22, FUB-PB-22, and their metabolites
___________________________________________________
Metabolic pathways of FDU-PB-22 and FUB-PB-22 in human hepatocytes
Metabolic pathways of FDU-PB-22 and FUB-PB-22 in human hepatocytes
___________________________________________________
Metabolic profiles of FDU-PB-22 and FUB-PB-22 in case urine 1 (a unhydrolyzed, b hydrolyzed) and case urine 2 (c unhydrolyzed, d hydrolyzed)
Metabolic profiles of FDU-PB-22 and FUB-PB-22 in case urine 1 (a unhydrolyzed, b hydrolyzed) and case urine 2 (c unhydrolyzed, d hydrolyzed)
__________

___________________________________________________
MRI of the pituitary at the time of initial presentation showing an indeterminate sellar/suprasellar heterogeneous mass measuring up to 2 cm (shown by arrows). A, Coronal section without contrast. B, Sagittal section without contrast. C, Coronal section with contrast demonstrating lack of enhancement. D, Sagittal section with contrast demonstrating lack of enhancement. There is extension to the optic chiasm. Original differential diagnosis per neuroradiology included an atypical pituitary macroadenoma, less likely craniopharyngioma, germ cell tumor, or metastasis. MRI = magnetic resonance imaging.
MRI of the pituitary at the time of initial presentation showing an indeterminate sellar/suprasellar heterogeneous mass measuring up to 2 cm (shown by arrows). A, Coronal section without contrast. B, Sagittal section without contrast. C, Coronal section with contrast demonstrating lack of enhancement. D, Sagittal section with contrast demons

In [15]:
print(len(cleanedDocs))
print(medDocs[56]['caption'])
print(cleanedDocs[56]['caption'])
# cleanedDocs[262]

59
['Adrenal radiation planning depicted on computed tomography scan of the abdomen. A, Planned target volume (in red) for adrenal SBRT included a 0.5 cm safety margin around the radiologic abnormality (in green). Organs at risk for damage from radiation treatment (OARs) included bilateral kidneys (purple), spinal cord (turquoise), stomach (magenta), bowel (blue), and liver (yellow). B, Image shows orientation of stereotactic radiotherapy beam arcs. One clockwise and one counterclockwise partial-arc arrangement was utilized to treat each target. C, The adrenal beds were treated to 3,900 cGy in 3 fractions. Dose color-wash at approximately 50% level shows rapid dose fall-off and excellent sparing of OARs. SBRT = stereotactic body radiotherapy.', 'Graph shows changes in serum cortisol and plasma ACTH levels over time. Treatment chronology included for reference. ACTH = adrenocortico-tropic hormone; SBRT = stereotactic body radiotherapy.']
[['adrenal', 'radiation', 'planning', 'depicted',

In [ ]:
# constructing inverted index
def getInvertedIdx(medDocs):
    invertedIdx = {}

    for i, doc in enumerate(medDocs):
        for term in doc['articleTitle']:
            if term in invertedIdx:
                invertedIdx[term].add((i,'t'))
            else:
                invertedIdx[term] = {(i,'t')}
        for term in doc['caption']:
            if term in invertedIdx:
                invertedIdx[term].add((i,'c'))
            else:
                invertedIdx[term] = {(i,'c')}
    return invertedIdx
           
termInvIdx = getInvertedIdx(cleanedDocs)
termInvIdx


In [ ]:
termInvIdx['biopsy']


In [ ]:
# Boolean operations:
def orPostings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    n1 = len(posting1)
    n2 = len(posting2)
    while p1 < n1 and p2 <n2:
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] >posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < n1:
        result.append(posting1[p1])
        p1 += 1
    while p2 < n2:
        result.append(posting2[p2])
        p2 += 1
    return result

def andPostings(posting1, posting2):
    p1 = 0
    p2 = 0
    n1 = len(posting1)
    n2 = len(posting2)
    result = list()
    
    while p1 < n1 and p2 < n2:
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
            
    return result 
        

In [ ]:

pl_1 = list(termInvIdx['biopsy'])
pl_2 = list(termInvIdx['tissues'])

#pl_3 = list(termInvIdx['histopatology'])
#pl_4 = list(termInvIdx['photomicrographs'])


pl_5 = list(termInvIdx['case'])
pl_6 = list(termInvIdx['report'])

orPostings(pl_1, pl_2)


In [ ]:

andPostings(pl_5, pl_6)

In [ ]:
orPostings(orPostings(pl_1, pl_2), andPostings(pl_5, pl_6))